<a href="https://colab.research.google.com/github/GeoKauko/TheNavySeals/blob/main/2_Seal_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-net seal detecion

## Import Libraries

In [1]:
#connect to the google drive if you use Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')
#install the libraries
!pip install rasterio
!pip install geopandas

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 38.0 MB/s eta 0:00:00


### Define path and load train and test data


In [ ]:
file_path = "/content/drive/MyDrive/Colab/Navy_seal"

In [ ]:
X_train =
Y_train =

X_test =
Y_test =


###Model generation (U-Net) and training

#### Check GPU

In [ ]:
#check the GPU colab assigns to you
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)